In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer



nltk.download('punk_tab')
nltk.download('stopwords')

[nltk_data] Error loading punk_tab: Package 'punk_tab' not found in
[nltk_data]     index
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\avina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam


In [91]:
import pandas as pd

data = pd.DataFrame({
    "tag": ["greeting", "goodbye", "thanks", "unknown"],
    "patterns": [
        "Hi;Hello;How are you?",
        "Bye;Goodbye;See you later",
        "Thanks;Thank you;Much appreciated",
        "What is this;I don't know;Tell me something"
    ],
    "responses": [
        "Hello! How can I help you?",
        "Goodbye! Have a great day!",
        "You're welcome! Let me know if you need anything else.",
        "Sorry, I didn't understand that. Could you please rephrase?"
    ]
})

data["patterns"] = data["patterns"].apply(lambda x: x.split(";"))
data = data.explode("patterns").reset_index(drop=True)

In [92]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    words = word_tokenize(text.lower())
    words = [w for w in words if w not in stop_words]
    return " ".join(words)

data["clean_text"] = data["patterns"].apply(clean_text)

# Encode labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data["encoded_tag"] = label_encoder.fit_transform(data["tag"])

# TF-IDF Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=500)
X = tfidf.fit_transform(data["clean_text"]).toarray()
Y = data["encoded_tag"].values

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\avina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\avina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [93]:
from tensorflow.keras.layers import Dropout


model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=30, batch_size=8, validation_data=(X_test, Y_test))


Epoch 1/30


C:\Users\avina\anaconda3\envs\NLP\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 227ms/step - accuracy: 0.1898 - loss: 1.3674 - val_accuracy: 0.0000e+00 - val_loss: 1.3986
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.2315 - loss: 1.3718 - val_accuracy: 0.0000e+00 - val_loss: 1.4052
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.1157 - loss: 1.4201 - val_accuracy: 0.0000e+00 - val_loss: 1.4111
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.1157 - loss: 1.3518 - val_accuracy: 0.0000e+00 - val_loss: 1.4189
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.1157 - loss: 1.4248 - val_accuracy: 0.0000e+00 - val_loss: 1.4245
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.1157 - loss: 1.4069 - val_accuracy: 0.0000e+00 - val_loss: 1.4303
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.4630 - loss: 1.3075 - val_accuracy: 0.0000e+00 - val_loss: 1.4371
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.5370 - loss: 1.2465 - val_accuracy: 0.0000e

In [94]:
import random

# Define output function
def output(tag):
        return random.choice(data[data["tag"] == tag]["responses"].tolist())
        

In [106]:
def chatbot_response(user_input):
    clean_input = clean_text(user_input)
    input_vector = tfidf.transform([clean_input]).toarray()

    predict_prob = model.predict(input_vector)[0]
    tag_index = np.argmax(predict_prob)
    tag = label_encoder.inverse_transform([tag_index])[0]

    return output(tag)


In [ ]:
print("chatbot is running... type quit for exit")

while True:
    user_input = input("you: ")
    if user_input.lower() == 'quit':
        print('goodbye!')
        break
    response = chatbot_response(user_input)
    print(f"chatbot: {user_input}")

# Test chat

chatbot is running... type quit for exit


you:  hi


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
chatbot: hi


In [71]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Initialize dataset
data = pd.DataFrame({
    "tag": ["greeting", "goodbye", "thanks", "unknown"],
    "patterns": [
        "Hi;Hello;How are you?",
        "Bye;Goodbye;See you later",
        "Thanks;Thank you;Much appreciated",
        "What is this;I don't know;Tell me something"
    ],
    "responses": [
        "Hello! How can I help you?",
        "Goodbye! Have a great day!",
        "You're welcome! Let me know if you need anything else.",
        "Sorry, I didn't understand that. Could you please rephrase?"
    ]
})

data["patterns"] = data["patterns"].apply(lambda x: x.split(";"))
data = data.explode("patterns").reset_index(drop=True)

# Preprocessing
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    words = word_tokenize(text.lower())
    words = [w for w in words if w not in stop_words]
    return " ".join(words)

data["clean_text"] = data["patterns"].apply(clean_text)

# Encode labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data["encoded_tag"] = label_encoder.fit_transform(data["tag"])

# TF-IDF Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=500)
X = tfidf.fit_transform(data["clean_text"]).toarray()
Y = data["encoded_tag"].values

# Train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Build TensorFlow model
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=30, batch_size=8, validation_data=(X_test, Y_test))

# Define output function
def output(tag):
    return random.choice(data[data["tag"] == tag]["responses"].tolist())

# Chatbot response function
def chatbot_response(user_input):
    clean_input = clean_text(user_input)
    input_vector = tfidf.transform([clean_input]).toarray()

    predict_prob = model.predict(input_vector)[0]
    tag_index = np.argmax(predict_prob)
    tag = label_encoder.inverse_transform([tag_index])[0]

    return output(tag)

# Test chatbot
print("Chatbot is running! Type 'quit' to exit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        print("Chatbot: Goodbye!")
        break
    response = chatbot_response(user_input)
    print(f"Chatbot: {response}")



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\avina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\avina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\avina\anaconda3\envs\NLP\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 553ms/step - accuracy: 0.0741 - loss: 1.4742 - val_accuracy: 0.0000e+00 - val_loss: 1.4024
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - accuracy: 0.3056 - loss: 1.3996 - val_accuracy: 0.0000e+00 - val_loss: 1.4084
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step - accuracy: 0.1157 - loss: 1.4838 - val_accuracy: 0.0000e+00 - val_loss: 1.4113
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.2315 - loss: 1.4391 - val_accuracy: 0.0000e+00 - val_loss: 1.4131
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - accuracy: 0.2315 - loss: 1.3500 - val_accuracy: 0.0000e+00 - val_loss: 1.4168
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.1898 - loss: 1.4323 - val_accuracy: 0.0000e+00 - val_loss: 1.4206
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - accuracy: 0.3056 - loss: 1.3560 - val_accuracy: 0.0000e+00 - val_loss: 1.4256
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.4630 - loss: 1.3608 - val_

You:  hi


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
Chatbot: You're welcome! Let me know if you need anything else.


You:  quit


Chatbot: Goodbye!
